In [ ]:
# sklearnex加速
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import sklearnex
sklearnex.patch_sklearn()

# 引用ARIMA模型
# 读取csv文件

df = pd.read_csv('data.csv', encoding='gbk').to_numpy()
df = df.astype('float64')


In [ ]:
# 数据归一化
max_value = np.max(df)
min_value = np.min(df)
scalar = max_value - min_value
df = list(map(lambda x: x / scalar, df))
df = np.array(df)


In [ ]:
# 将数据集分为训练集和测试集
train = df[:int(0.95*(len(df)))]
test = df[int(0.95*(len(df))):]

In [ ]:
plt.plot(train)

In [ ]:
# 用ARIMA进行预测
# order=(p,d,q) p:自回归阶数 d:差分阶数 q:移动平均阶数
model = ARIMA(train, order=(3, 3, 30))

model_fit = model.fit()

# 预测测试集
ARM_yhat = model_fit.predict(len(train), len(train)+len(test)-2, typ='levels')


In [ ]:
windows = 7
matrix = np.zeros((df.size-windows, windows+1))
for i in range(df.size-windows):    # 生成矩阵
    matrix[i] = df[i:i+windows+1].T
# 将数据集分为训练集和测试集
train_m = matrix[:int(0.95*(len(matrix)))]
test_m = matrix[int(0.95*(len(matrix))):]


In [ ]:
# 用XGBoost进行预测
# 将数据格式转换成DMatrix格式
train_dmatrix = xgb.DMatrix(data=train_m[:, :-1], label=train_m[:, -1])
test_dmatrix = xgb.DMatrix(data=test_m[:, :-1], label=test_m[:, -1])

# 设置参数
params = {'booster': 'gbtree',    # 使用gbtree,可选项gbtree,gbliner
          'objective': 'reg:linear',  # 使用线性回归,可选项reg:linear,reg:logistic
          'eval_metric': 'rmse',  # 评估函数,回归问题rmse,分类问题error
          'gamma': 0.1,  # 指定了节点分裂所需的最小损失函数下降值
          'min_child_weight': 1.1,   # 指定了叶子节点最小的样本权重和
          'max_depth': 500,  # 树的最大深度
          'lambda': 10,   # 权重的L2正则化项
          'subsample': 0.5,   # 训练每棵树时使用的数据占全部训练集的比例
          'colsample_bytree': 0.5,   # 训练每棵树时使用的特征占全部特征的比例
          'colsample_bylevel': 0.5,   # 训练每棵树时使用的特征占全部特征的比例
          'eta': 0.1,     # 学习率
          'tree_method': 'auto',  # 使用的建树方法,可选项exact,approx,auto
          'seed': 0,
          'nthread': 12  # 并行线程数
          }

# 训练模型
model = xgb.train(params, train_dmatrix, num_boost_round=1000, evals=[
                  (test_dmatrix, 'test')], early_stopping_rounds=100)

# 预测测试集
xgbyhat = model.predict(test_dmatrix, ntree_limit=model.best_ntree_limit)

# 绘图
plt.plot(xgbyhat, label='xgb_predict')
# plt.plot(ARM_yhat, label='ARM_predict')
plt.plot(test_m[:, -1], label='true')
plt.text(0, 0.5, 'RMSE=%.4f' % np.sqrt(mean_squared_error(
    test_m[:, -1], xgbyhat)))
plt.legend()
plt.show()


In [ ]:
# 用支持向量机进行预测
from sklearn.svm import SVR
# 将数据格式转换成DMatrix格式
train_dmatrix = xgb.DMatrix(data=train_m[:, :-1], label=train_m[:, -1])
test_dmatrix = xgb.DMatrix(data=test_m[:, :-1], label=test_m[:, -1])

# 设置参数
params = {'kernel': 'rbf',    # 核函数,可选项rbf,poly,sigmoid,linear
          'gamma': 'auto',    # 核函数系数,可选项scale,auto
          'C': 1.0,   # 惩罚系数
          'epsilon': 0.1,     # 指定了支持向量机回归模型的损失函数中的ε
          'shrinking': True,  # 是否使用启发式
          'tol': 0.001,   # 指定了支持向量机算法停止的容忍度
          'cache_size': 2000,  # 指定了核矩阵的缓存大小
          'verbose': False,   # 是否启用详细输出
          'max_iter': -1,     # 指定了最大迭代次数
          }

# 训练模型
model = SVR(**params)
model.fit(train_m[:, :-1], train_m[:, -1])

# 预测测试集
svr_yhat = np.zeros((test_m.shape[0], 1))
# 预测测试集
for i in range(test_m.shape[0]):
    svr_yhat[i] = model.predict(test_m[i:i+1, :-1].reshape(1, -1))
    test_m[i+1:i+2, -2:-1]= svr_yhat[i]
# 绘图
plt.plot(xgbyhat, label='xgb_predict')
plt.plot(svr_yhat, label='svr_predict')
#plt.plot(ARM_yhat, label='ARM_predict')
plt.plot(test_m[:, -1], label='true')
plt.text(0, 0.5, 'XGB-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], xgbyhat)))
plt.text(0, 0.4, 'SVR-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], svr_yhat)))
#plt.text(0, 0.3, 'ARM-RMSE: %.3f' %
         #np.sqrt(mean_squared_error(test_m[:, -1], ARM_yhat)))
plt.legend()
plt.show()


In [ ]:
# 用随机森林进行预测
from sklearn.ensemble import RandomForestRegressor
# 将数据格式转换成DMatrix格式
train_dmatrix = xgb.DMatrix(data=train_m[:, :-1], label=train_m[:, -1])
test_dmatrix = xgb.DMatrix(data=test_m[:, :-1], label=test_m[:, -1])

# 设置参数
params = {'n_estimators': 100,    # 森林中树的数量
          # 分裂节点时评估准则,可选项'poisson', 'squared_error', 'absolute_error', 'friedman_mse'
          'criterion': 'squared_error',
          'max_depth': None,  # 树的最大深度
          'min_samples_leaf': 1,  # 叶子节点所需的最小样本数
          'min_weight_fraction_leaf': 0.0,    # 叶子节点所需的最小加权分数
          'max_features': 'auto',     # 寻找最佳分割时考虑的特征数,可选项auto,sqrt,log2
          'max_leaf_nodes': None,     # 叶子节点的最大数量
          'min_impurity_decrease': 0.0,    # 如果节点分裂会导致杂质的减少大于或等于该值,则分裂该节点
          'bootstrap': True,  # 是否使用bootstrap样本
          'oob_score': False,     # 是否使用袋外样本来估计泛化精度
          'n_jobs': None,     # 并行运行的作业数
          'random_state': None,   # 随机种子
          'verbose': 0,   # 是否启用详细输出
          'warm_start': False,    # 是否使用上一次调用该类的结果然后增加新的树
          'ccp_alpha': 0.0,   # 非负的复杂性参数
          'max_samples': None     # 用于训练每棵树的样本数
          }

# 训练模型
model = RandomForestRegressor(**params)
model.fit(train_m[:, :-1], train_m[:, -1])


rf_yhat = np.zeros((test_m.shape[0], 1))
# 预测测试集
for i in range(test_m.shape[0]):
    rf_yhat[i] = model.predict(test_m[i:i+1, :-1].reshape(1, -1))
    test_m[i+1:i+2, -2:-1]= rf_yhat[i]

# 绘图
plt.plot(xgbyhat, label='xgb_predict')
plt.plot(svr_yhat, label='svr_predict')
plt.plot(rf_yhat, label='rf_predict')
# plt.plot(ARM_yhat, label='ARM_predict')
plt.plot(test_m[:, -1], label='true')
plt.text(0, 0.5, 'XGB-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], xgbyhat)))
plt.text(0, 0.4, 'SVR-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], svr_yhat)))
plt.text(0, 0.3, 'RF-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], rf_yhat)))
# plt.text(0, 0.2, 'ARM-RMSE: %.3f' % np.sqrt(mean_squared_error(test_m[:, -1], ARM_yhat)))
plt.legend()
plt.show()


In [ ]:
# 用KNN进行预测
from sklearn.neighbors import KNeighborsRegressor
# 将数据格式转换成DMatrix格式
train_dmatrix = xgb.DMatrix(data=train_m[:, :-1], label=train_m[:, -1])
test_dmatrix = xgb.DMatrix(data=test_m[:, :-1], label=test_m[:, -1])

# 设置参数
params = {'n_neighbors': 5,   # 邻居数
          'weights': 'distance',   # 权重函数,可选项uniform,distance
          'algorithm': 'auto',    # 计算最近邻居的算法,可选项auto,ball_tree,kd_tree,brute
          'leaf_size': 30,    # 传递给BallTree或KDTree的叶子大小
          'p': 2,     # 闵可夫斯基距离参数，p = 1为曼哈顿距离，p = 2为欧式距离
          'metric': 'minkowski',  # 用于树的距离度量，可选项minkowski,manhattan,euclidean,chebyshev
          'metric_params': None,  # 距离度量的其他关键字参数
          'n_jobs': None  # 并行运行的作业数
          }

# 训练模型
model = KNeighborsRegressor(**params)
model.fit(train_m[:, :-1], train_m[:, -1])

knn_yhat = np.zeros((test_m.shape[0], 1))
# 预测测试集
for i in range(test_m.shape[0]):
    knn_yhat[i] = model.predict(test_m[i:i+1, :-1].reshape(1, -1))
    test_m[i+1:i+2, -2:-1]= knn_yhat[i]

# 绘图
# plt.plot(xgbyhat, label='xgb_predict')
# plt.plot(svr_yhat, label='svr_predict')
# plt.plot(rf_yhat, label='rf_predict')
plt.plot(knn_yhat, label='knn_predict')
# plt.plot(ARM_yhat, label='ARM_predict')
plt.plot(test_m[:, -1], label='true')
plt.text(0, 0.5, 'XGB-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], xgbyhat)))
plt.text(0, 0.4, 'SVR-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], svr_yhat)))
plt.text(0, 0.3, 'RF-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], rf_yhat)))
plt.text(0, 0.2, 'KNN-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], knn_yhat)))
# plt.text(0, 0.1, 'ARM-RMSE: %.3f' % np.sqrt(mean_squared_error(test_m[:, -1], ARM_yhat)))
plt.legend()
plt.show()


In [ ]:
# 用决策树进行预测
from sklearn.tree import DecisionTreeRegressor

# 设置参数
params = {'criterion': 'absolute_error',  # 分裂节点时评估准则,可选项'poisson', 'squared_error', 'absolute_error', 'friedman_mse'
          'splitter': 'best',     # 分裂节点的策略,可选项'best','random'
          'max_depth': None,  # 树的最大深度
          'min_samples_leaf': 1,  # 叶子节点所需的最小样本数
          'min_weight_fraction_leaf': 0.0,    # 叶子节点所需的最小加权分数
          'max_features': None,   # 寻找最佳分割时考虑的特征数,可选项auto,sqrt,log2
          'max_leaf_nodes': None,     # 叶子节点的最大数量
          'min_impurity_decrease': 0.0,    # 如果节点分裂会导致杂质的减少大于或等于该值,则分裂该节点
          'random_state': None,   # 随机种子
          }

# 训练模型
model = DecisionTreeRegressor(**params)
model.fit(train_m[:, :-1], train_m[:, -1])

# 预测测试集
dt_yhat = np.zeros((test_m.shape[0], 1))
# 预测测试集
for i in range(test_m.shape[0]):
    dt_yhat[i] = model.predict(test_m[i:i+1, :-1].reshape(1, -1))
    test_m[i+1:i+2, -2:-1]= dt_yhat[i]

# 绘图
# plt.plot(rf_yhat, label='rf_predict')
plt.plot(dt_yhat, label='dt_predict')
# plt.plot(ARM_yhat, label='ARM_predict')
plt.plot(test_m[:, -1], label='true')
plt.text(0, 0.5, 'RF-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], rf_yhat)))
plt.text(0, 0.4, 'KNN-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], knn_yhat)))
plt.text(0, 0.3, 'DT-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], dt_yhat)))
plt.legend()
plt.show()


In [ ]:
# 用线性回归进行预测
from sklearn.linear_model import LinearRegression

# 设置参数
params = {'fit_intercept': True,  # 是否计算截距
          'copy_X': True,     # 是否复制X
          'n_jobs': None  # 并行运行的作业数
          }

# 训练模型
model = LinearRegression(**params)
model.fit(train_m[:, :-1], train_m[:, -1])

# 预测测试集
lr_yhat=np.zeros((test_m.shape[0], 1))
for i in range(test_m.shape[0]):
    lr_yhat[i] = model.predict(test_m[i:i+1, :-1].reshape(1, -1))
    test_m[i+1:i+2, -2:-1]= lr_yhat[i]

# 绘图
# plt.plot(rf_yhat, label='rf_predict')
plt.plot(lr_yhat, label='lr_predict')
# plt.plot(knn_yhat, label='knn_predict')
plt.plot(test_m[:, -1], label='true')
plt.text(0, 0.5, 'RF-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], rf_yhat)))
plt.text(0, 0.4, 'KNN-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], knn_yhat)))
plt.text(0, 0.3, 'LR-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], lr_yhat)))


In [ ]:
# 用多项式回归进行预测
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

# 设置参数
params = {'degree': 2,     # 多项式的次数
          'interaction_only': False,  # 是否只包含交互项
          'include_bias': True   # 是否包含偏差
          }

# 训练模型
model = Pipeline([('poly', PolynomialFeatures(**params)),
                 ('linear', LinearRegression())])
model.fit(train_m[:, :-1], train_m[:, -1])

pr_yhat = np.zeros((test_m.shape[0], 1))
# 预测测试集
for i in range(test_m.shape[0]):
    pr_yhat[i] = model.predict(test_m[i:i+1, :-1].reshape(1, -1))
    test_m[i+1:i+2, -2:-1]= pr_yhat[i]

# 绘图
# plt.plot(rf_yhat, label='rf_predict')
plt.plot(pr_yhat, label='pr_predict')
# plt.plot(knn_yhat, label='knn_predict')
plt.plot(test_m[:, -1], label='true')
plt.text(0, 0.5, 'RF-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], rf_yhat)))
plt.text(0, 0.4, 'KNN-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], knn_yhat)))
plt.text(0, 0.3, 'PR-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], pr_yhat)))
plt.legend()
plt.show()


In [ ]:
# 用岭回归进行预测
from sklearn.linear_model import Ridge

# 设置参数
params = {'alpha': 1.0,   # 正则化强度
          'fit_intercept': True,  # 是否计算截距
          'copy_X': True,     # 是否复制X
          'max_iter': None,   # 最大迭代次数
          'tol': 0.001,   # 指定了支持向量机算法停止的容忍度
          'solver': 'auto',   # 求解器,可选项auto,sag,saga,sparse_cg,lsqr,saga
          'random_state': None    # 随机种子
          }

# 训练模型
model = Ridge(**params)
model.fit(train_m[:, :-1], train_m[:, -1])
rd_yhat = np.zeros((test_m.shape[0], 1))
# 预测测试集
for i in range(test_m.shape[0]):
    rd_yhat[i] = model.predict(test_m[i:i+1, :-1].reshape(1, -1))
    test_m[i+1:i+2, -2:-1]= rd_yhat[i]

# 绘图
# plt.plot(rf_yhat, label='rf_predict')
plt.plot(rd_yhat, label='rd_predict')
# plt.plot(knn_yhat, label='knn_predict')
plt.plot(test_m[:, -1], label='true')
plt.text(0, 0.5, 'RF-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], rf_yhat)))
plt.text(0, 0.4, 'KNN-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], knn_yhat)))
plt.text(0, 0.3, 'RD-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], rd_yhat)))
plt.legend()
plt.show()


In [ ]:
# 用LSTM进行预测
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# 设置参数
params = {'units': 50,    # 输出空间的维度
          # 激活函数,可选项softmax,softplus,softsign,relu,tanh,sigmoid,hard_sigmoid,linear
          'activation': 'tanh',
          # 递归层激活函数,可选项softmax,softplus,softsign,relu,tanh,sigmoid,hard_sigmoid,linear
          'recurrent_activation': 'sigmoid',
          'use_bias': True,   # 是否使用偏差
          # 权重矩阵的初始化方法,可选项glorot_uniform,lecun_uniform,normal,orthogonal,zero,one
          'kernel_initializer': 'glorot_uniform',
          # 递归矩阵的初始化方法,可选项glorot_uniform,lecun_uniform,normal,orthogonal,zero,one
          'recurrent_initializer': 'orthogonal',
          # 偏差向量的初始化方法,可选项glorot_uniform,lecun_uniform,normal,orthogonal,zero,one
          'bias_initializer': 'zeros',
          'unit_forget_bias': True,   # 是否添加1到偏差
          'dropout': 0.0,     # 输入单元的dropout比例
          'recurrent_dropout': 0.0,   # 递归单元的dropout比例
          'implementation': 1,    # 实现模式,可选项1,2
          'return_sequences': False,  # 是否返回全部输出序列
          'return_state': False,  # 是否返回最后一个状态
          'go_backwards': False,  # 是否反向处理输入序列
          'stateful': False,  # 是否在批次间保持递归状态
          'unroll': False    # 是否展开递归层
          }

# 训练模型
model = Sequential() 
model.add(LSTM(**params, input_shape=( windows,1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_m[:, :-1], train_m[:, -1],
          epochs=100, batch_size=64, verbose=2)
lstm_yhat = np.zeros((test_m.shape[0], 1))
# 预测测试集
for i in range(test_m.shape[0]):
    lstm_yhat[i] = model.predict(test_m[i:i+1, :-1].reshape(1, -1))
    test_m[i+1:i+2, -2:-1]= lstm_yhat[i]

# 绘图 
plt.plot(lstm_yhat, label='lstm_predict')
plt.plot(test_m[:, -1], label='true')
plt.text(0, 0.5, 'LSTM-RMSE: %.3f' %
            np.sqrt(mean_squared_error(test_m[:, -1], lstm_yhat)))
plt.legend()
plt.show()


In [ ]:
# 用GRU进行预测
from keras.layers import GRU

# 设置参数
params = {'units': 50,    # 输出空间的维度
          # 激活函数,可选项softmax,softplus,softsign,relu,tanh,sigmoid,hard_sigmoid,linear
          'activation': 'tanh',
          # 递归层激活函数,可选项softmax,softplus,softsign,relu,tanh,sigmoid,hard_sigmoid,linear
          'recurrent_activation': 'sigmoid',
          'use_bias': True,   # 是否使用偏差
          # 权重矩阵的初始化方法,可选项glorot_uniform,lecun_uniform,normal,orthogonal,zero,one
          'kernel_initializer': 'zero',
          # 递归矩阵的初始化方法,可选项glorot_uniform,lecun_uniform,normal,orthogonal,zero,one
          'recurrent_initializer': 'zero',
          # 偏差向量的初始化方法,可选项glorot_uniform,lecun_uniform,normal,orthogonal,zero,one
          'bias_initializer': 'zeros',
          'kernel_regularizer': None,     # 权重矩阵的正则化方法,可选项l1,l2,l1_l2
          'recurrent_regularizer': None,  # 递归矩阵的正则化方法,可选项l1,l2,l1_l2
          'bias_regularizer': None,   # 偏差向量的正则化方法,可选项l1,l2,l1_l2
          'activity_regularizer': None,   # 输出的正则化方法,可选项l1,l2,l1_l2
          'kernel_constraint': None,  # 权重矩阵的约束方法,可选项max_norm,non_neg
          'recurrent_constraint': None,   # 递归矩阵的约束方法,可选项max_norm,non_neg
          'bias_constraint': None,    # 偏差向量的约束方法,可选项max_norm,non_neg
          'dropout': 0.0,     # 输入单元的dropout比例
          'recurrent_dropout': 0.0,   # 递归单元的dropout比例
          'implementation': 1,    # 实现模式,可选项1,2
          'return_sequences': False,  # 是否返回全部输出序列
          'return_state': False,  # 是否返回最后一个状态
          'go_backwards': False,  # 是否反向处理输入序列
          'stateful': False,  # 是否在批次间保持递归状态
          'unroll': False    # 是否展开递归层
          }

# 训练模型
model = Sequential()
model.add(GRU(**params, input_shape=(train_m.shape[1]-1, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_m[:, :-1], train_m[:, -1],
          epochs=100, batch_size=64, verbose=2)
gru_yhat = np.zeros((test_m.shape[0], 1))
# 预测测试集
for i in range(test_m.shape[0]):
    gru_yhat[i] = model.predict(test_m[i:i+1, :-1].reshape(1, -1))
    test_m[i+1:i+2, -2:-1]= gru_yhat[i]

# 绘图
# plt.plot(rf_yhat, label='rf_predict')
plt.plot(gru_yhat, label='gru_predict')
#plt.plot(knn_yhat, label='knn_predict')
plt.plot(test_m[:, -1], label='true')
plt.text(0, 0.5, 'RF-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], rf_yhat)))
plt.text(0, 0.4, 'KNN-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], knn_yhat)))
plt.text(0, 0.3, 'GRU-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], gru_yhat)))
plt.legend()
plt.show()


In [ ]:
0.045*scalar

In [ ]:
# 用CNN进行预测
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten

# 设置参数
params = {'filters': 64,   # 卷积核的数量
          'kernel_size': 3,   # 卷积核的大小
          'strides': 1,   # 卷积核的步长
          'padding': 'same',  # 填充方式,可选项valid,same
          'data_format': 'channels_last',     # 输入数据的格式,可选项channels_last,channels_first
          'dilation_rate': 1,     # 卷积核的膨胀率
          # 激活函数,可选项softmax,softplus,softsign,relu,tanh,sigmoid,hard_sigmoid,linear
          'activation': 'relu',
          'use_bias': True,   # 是否使用偏差
          # 权重矩阵的初始化方法,可选项glorot_uniform,lecun_uniform,normal,orthogonal,zero,one
          'kernel_initializer': 'glorot_uniform',
          # 偏差向量的初始化方法,可选项glorot_uniform,lecun_uniform,normal,orthogonal,zero,one
          'bias_initializer': 'zeros'
          }

# 训练模型
model = Sequential()
model.add(Conv1D(**params, input_shape=(train_m.shape[1]-1, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_m[:, :-1], train_m[:, -1],
          epochs=100, batch_size=64, verbose=2)

# 预测测试集
cnn_yhat=np.zeros((test_m.shape[0], 1))
for i in range(test_m.shape[0]):
    cnn_yhat[i] = model.predict(test_m[i:i+1, :-1].reshape(1, -1))
    test_m[i+1:i+2, -2:-1]= cnn_yhat[i]

# 绘图
# plt.plot(rf_yhat, label='rf_predict')
plt.plot(cnn_yhat, label='cnn_predict')
plt.plot(knn_yhat, label='knn_predict')
plt.plot(test_m[:, -1], label='true')
plt.text(0, 0.5, 'RF-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], rf_yhat)))
plt.text(0, 0.4, 'KNN-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], knn_yhat)))
plt.text(0, 0.3, 'CNN-RMSE: %.3f' %
         np.sqrt(mean_squared_error(test_m[:, -1], cnn_yhat)))
plt.legend()
plt.show()


In [ ]:
#用RNN进行预测 
from keras.layers import SimpleRNN

# 设置参数
params = {'units': 50,    # 输出空间的维度
            # 激活函数,可选项softmax,softplus,softsign,relu,tanh,sigmoid,hard_sigmoid,linear
            'activation': 'tanh',
            'use_bias': True,   # 是否使用偏差
            'dropout': 0.0,     # 输入单元的dropout比例
            'recurrent_dropout': 0.0,   # 递归单元的dropout比例
            'return_sequences': False,  # 是否返回全部输出序列
            'return_state': False,  # 是否返回最后一个状态
            'go_backwards': False,  # 是否反向处理输入序列
            'stateful': False,  # 是否在批次间保持递归状态
            'unroll': False    # 是否展开递归层
            }

# 训练模型
model = Sequential()
model.add(SimpleRNN(**params, input_shape=(train_m.shape[1]-1, 1)))
model.add(Dense(1)) 
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_m[:, :-1], train_m[:, -1],
            epochs=100, batch_size=64, verbose=2)

# 预测测试集
rnn_yhat = model.predict(test_m[:, :-1]) 

# 绘图
# plt.plot(rf_yhat, label='rf_predict')
plt.plot(rnn_yhat, label='rnn_predict')
plt.plot(knn_yhat, label='knn_predict')
plt.plot(test_m[:, -1], label='true')
plt.text(0, 0.5, 'RF-RMSE: %.3f' % np.sqrt(mean_squared_error(test_m[:, -1], rf_yhat)))
plt.text(0, 0.4, 'KNN-RMSE: %.3f' % np.sqrt(mean_squared_error(test_m[:, -1], knn_yhat)))
plt.text(0, 0.3, 'RNN-RMSE: %.3f' % np.sqrt(mean_squared_error(test_m[:, -1], rnn_yhat)))
plt.legend()
plt.show()


In [ ]:
#用BiLSTM进行预测 
from keras.layers import Bidirectional

# 设置参数
params = {'units': 50,    # 输出空间的维度
            # 激活函数,可选项softmax,softplus,softsign,relu,tanh,sigmoid,hard_sigmoid,linear
            'activation': 'tanh',
            # 递归层激活函数,可选项softmax,softplus,softsign,relu,tanh,sigmoid,hard_sigmoid,linear
            'recurrent_activation': 'sigmoid',
            'use_bias': True,   # 是否使用偏差
            # 权重矩阵的初始化方法,可选项glorot_uniform,lecun_uniform,normal,orthogonal,zero,one
            'kernel_initializer': 'glorot_uniform',
            # 递归矩阵的初始化方法,可选项glorot_uniform,lecun_uniform,normal,orthogonal,zero,one
            'recurrent_initializer': 'orthogonal',
            # 偏差向量的初始化方法,可选项glorot_uniform,lecun_uniform,normal,orthogonal,zero,one
            'bias_initializer': 'zeros',
            'unit_forget_bias': True,   # 是否添加1到偏差
            'dropout': 0.0,     # 输入单元的dropout比例
            'recurrent_dropout': 0.0,   # 递归单元的dropout比例
            'implementation': 1,    # 实现模式,可选项1,2
            'return_sequences': False,  # 是否返回全部输出序列
            'return_state': False,  # 是否返回最后一个状态
            'go_backwards': False,  # 是否反向处理输入序列
            'stateful': False,  # 是否在批次间保持递归状态
            'unroll': False    # 是否展开递归层
            }

# 训练模型
model = Sequential()
model.add(Bidirectional(LSTM(**params), input_shape=(train_m.shape[1]-1, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_m[:, :-1], train_m[:, -1],
            epochs=100, batch_size=64, verbose=2)

# 预测测试集
bilstm_yhat = model.predict(test_m[:, :-1])

# 绘图
# plt.plot(rf_yhat, label='rf_predict')
plt.plot(bilstm_yhat, label='bilstm_predict')
#plt.plot(knn_yhat, label='knn_predict')
plt.plot(test_m[:, -1], label='true')
plt.text(0, 0.5, 'RF-RMSE: %.3f' % np.sqrt(mean_squared_error(test_m[:, -1], rf_yhat)))
plt.text(0, 0.4, 'KNN-RMSE: %.3f' % np.sqrt(mean_squared_error(test_m[:, -1], knn_yhat)))
plt.text(0, 0.3, 'BiLSTM-RMSE: %.3f' % np.sqrt(mean_squared_error(test_m[:, -1], bilstm_yhat)))
plt.legend()
plt.show()

In [ ]:
#用BiGRU进行预测
from keras.layers import Bidirectional

# 设置参数
params = {'units': 50,    # 输出空间的维度
            # 激活函数,可选项softmax,softplus,softsign,relu,tanh,sigmoid,hard_sigmoid,linear
            'activation': 'tanh',
            # 递归层激活函数,可选项softmax,softplus,softsign,relu,tanh,sigmoid,hard_sigmoid,linear
            'recurrent_activation': 'sigmoid',
            'use_bias': True,   # 是否使用偏差
            'dropout': 0.0,     # 输入单元的dropout比例
            'recurrent_dropout': 0.0,   # 递归单元的dropout比例
            'implementation': 1,    # 实现模式,可选项1,2
            'return_sequences': False,  # 是否返回全部输出序列
            'return_state': False,  # 是否返回最后一个状态
            'go_backwards': False,  # 是否反向处理输入序列
            'stateful': False,  # 是否在批次间保持递归状态
            'unroll': False    # 是否展开递归层
            }

# 训练模型
model = Sequential() 
model.add(Bidirectional(GRU(**params), input_shape=(train_m.shape[1]-1, 1))) 
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_m[:, :-1], train_m[:, -1],
            epochs=100, batch_size=64, verbose=2)

# 预测测试集
bigru_yhat = model.predict(test_m[:, :-1])

# 绘图
# plt.plot(rf_yhat, label='rf_predict')
plt.plot(bigru_yhat, label='bigru_predict')
#plt.plot(knn_yhat, label='knn_predict')
plt.plot(test_m[:, -1], label='true')
plt.text(0, 0.5, 'RF-RMSE: %.3f' % np.sqrt(mean_squared_error(test_m[:, -1], rf_yhat)))
plt.text(0, 0.4, 'KNN-RMSE: %.3f' % np.sqrt(mean_squared_error(test_m[:, -1], knn_yhat)))
plt.text(0, 0.3, 'BiGRU-RMSE: %.3f' % np.sqrt(mean_squared_error(test_m[:, -1], bigru_yhat)))
plt.legend()
plt.show()


![](1.jpg)

In [ ]:
#用ARIMA进行预测，再用BiLSTM预测残差，最后将预测值相加
model = ARIMA(train, order=(3, 3, 30))

model_fit = model.fit()

# 预测测试集
ARM_yhat = model_fit.predict(1, len(train), typ='levels')


# 计算残差
residual = train[:, -1] - ARM_yhat

residual_m = np.zeros((residual.size-windows, windows+1))
for i in range(residual.size-windows):    # 生成矩阵
    residual_m[i] = residual[i:i+windows+1].T


# 设置参数
params = {'units': 50,    # 输出空间的维度
            # 激活函数,可选项softmax,softplus,softsign,relu,tanh,sigmoid,hard_sigmoid,linear
            'activation': 'tanh',
            # 递归层激活函数,可选项softmax,softplus,softsign,relu,tanh,sigmoid,hard_sigmoid,linear
            'recurrent_activation': 'sigmoid',
            'use_bias': True,   # 是否使用偏差
            # 权重矩阵的初始化方法,可选项glorot_uniform,lecun_uniform,normal,orthogonal,zero,one
            'kernel_initializer': 'glorot_uniform',
            # 递归矩阵的初始化方法,可选项glorot_uniform,lecun_uniform,normal,orthogonal,zero,one
            'recurrent_initializer': 'orthogonal',
            # 偏差向量的初始化方法,可选项glorot_uniform,lecun_uniform,normal,orthogonal,zero,one
            'bias_initializer': 'zeros',
            'unit_forget_bias': True,   # 是否添加1到偏差
            'dropout': 0.0,     # 输入单元的dropout比例
            'recurrent_dropout': 0.0,   # 递归单元的dropout比例
            'implementation': 1,    # 实现模式,可选项1,2
            'return_sequences': False,  # 是否返回全部输出序列
            'return_state': False,  # 是否返回最后一个状态
            'go_backwards': False,  # 是否反向处理输入序列
            'stateful': False,  # 是否在批次间保持递归状态
            'unroll': False    # 是否展开递归层
            }

# 训练模型
model = Sequential()
model.add(Bidirectional(LSTM(**params), input_shape=(residual_m.shape[1]-1, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(residual_m[:, :-1], residual_m[:, -1],
          epochs=100, batch_size=64, verbose=2)

# 预测测试集

BiLSTM_yhat = np.zeros((test_m.shape[0], 1))

ARM_ypre=model_fit.predict(len(train)+1, len(train)+len(test)-1, typ='levels')

for i in range(test_m.shape[0]):
    BiLSTM_yhat[i] = model.predict(residual_m[i:i+1, :-1].reshape(1, -1))
    residual_m[i+1:i+2, -2:-1]= BiLSTM_yhat[i]

Total_yhat=ARM_ypre+BiLSTM_yhat.reshape(-1)

# 绘图 
plt.plot(Total_yhat, label='Total_predict')
plt.plot(test_m[:, -1], label='true')
plt.text(0, 0.5, 'Total-RMSE: %.3f' %
            np.sqrt(mean_squared_error(test_m[:, -1], Total_yhat)))
plt.legend()
plt.show()